In [1]:
import geopandas as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
json = gp.read_file('cb_2016_06_puma10_500k.json')

In [3]:
json['internet'] = np.array(alameda_internet)

NameError: name 'alameda_internet' is not defined

In [4]:
json.to_file('alameda.json', driver='GeoJSON')

DriverIOError: GeoJSON driver doesn't support creating a layer on a read-only datasource

In [5]:
# PUMAs shape files
pumas = gp.read_file('cb_2016_06_puma10_500k/cb_2016_06_puma10_500k.shp')
# read in household PUMS data
hca = pd.read_csv('ss16hca.csv')

## Get Internet Access Data

In [6]:
# extract samples corresponding to each county
alameda_serial = np.arange(101, 111)
alameda = hca.loc[hca.PUMA.isin(alameda_serial)]

In [7]:
def extend_low_income(thres, upper_N=16):
    extras = (np.arange(9, upper_N) - 8) * thres[3] * 0.08 + thres[-1]
    thres = np.append(thres, extras)
    return thres

In [9]:
alameda_low_income_thres = np.array([62750, 71700, 80650, 89600, 96800, 103950, 111150, 118300])
# extend the low income thresholds for households of size >8
alameda_low_income_thres = extend_low_income(alameda_low_income_thres)

In [10]:
def get_low_income(df, low_income_thresholds):
    # use FINCP or HINCP?
    # remove NaN household incomes
    trimmed = df.loc[df.FINCP.notna()]
    low_income = trimmed.loc[trimmed.FINCP < low_income_thresholds[trimmed.NP - 1]]
    return low_income

In [11]:
alameda_low_income = get_low_income(alameda, alameda_low_income_thres)

In [27]:
def calculate_percent_internet(table, weighted=False):
    grouped = table.groupby('PUMA')
    if weighted:
        weighted_sum = grouped.WGTP.sum()
        no_internet = table.loc[table.ACCESS == 3].groupby('PUMA').WGTP.sum()
        percent_no_internet = 100 * no_internet/weighted_sum
    else:
        no_internet = grouped.loc[table.ACCESS==3].agg('mean')
        percent_no_internet = 100 * no_internet
    return percent_no_internet

In [28]:
alameda_internet = calculate_percent_internet(alameda_low_income)

AttributeError: Cannot access callable attribute 'loc' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [26]:
alameda_internet

,SERIALNO,ST,DIVISION,REGION,ADJHSG,ADJINC,WGTP,NP,TYPE,ACCESS,...,wgtp71,wgtp72,wgtp73,wgtp74,wgtp75,wgtp76,wgtp77,wgtp78,wgtp79,wgtp80
PUMA,,,,,,,,,,,,,,,,,,,,,
101,1.304666e+08,600.0,900.0,400.0,100000000.0,100758800.0,9200.000000,200.000000,100.0,300.0,...,2700.000000,9800.000000,11200.000000,14700.000000,9000.000000,12900.000000,12900.000000,2500.000000,2800.000000,7300.000000
102,6.603780e+07,600.0,900.0,400.0,100000000.0,100758800.0,10509.090909,290.909091,100.0,300.0,...,10145.454545,11218.181818,9690.909091,7869.696970,9972.727273,9930.303030,12866.666667,9203.030303,11163.636364,9727.272727
103,6.498473e+07,600.0,900.0,400.0,100000000.0,100758800.0,10685.714286,271.428571,100.0,300.0,...,12671.428571,11285.714286,11314.285714,10728.571429,12342.857143,8385.714286,9028.571429,10428.571429,10342.857143,9185.714286
104,8.319246e+07,600.0,900.0,400.0,100000000.0,100758800.0,11246.938776,391.836735,100.0,300.0,...,10969.387755,10822.448980,11061.224490,10932.653061,12306.122449,11328.571429,10528.571429,10808.163265,12126.530612,11522.448980
105,8.245407e+07,600.0,900.0,400.0,100000000.0,100758800.0,14700.000000,250.000000,100.0,300.0,...,11050.000000,16700.000000,16321.428571,10507.142857,14664.285714,14621.428571,13971.428571,15657.142857,14707.142857,13021.428571
106,8.631878e+07,600.0,900.0,400.0,100000000.0,100758800.0,14707.692308,276.923077,100.0,300.0,...,13253.846154,17476.923077,18653.846154,11230.769231,10623.076923,11569.230769,12769.230769,16807.692308,16284.615385,14900.000000
107,6.967378e+07,600.0,900.0,400.0,100000000.0,100758800.0,10786.956522,295.652174,100.0,300.0,...,9456.521739,13400.000000,12673.913043,13078.260870,10308.695652,12308.695652,11765.217391,8695.652174,12565.217391,8652.173913
108,7.960223e+07,600.0,900.0,400.0,100000000.0,100758800.0,10353.333333,380.000000,100.0,300.0,...,11100.000000,12006.666667,10533.333333,8966.666667,9553.333333,11340.000000,10533.333333,9400.000000,10713.333333,9626.666667
109,7.345981e+07,600.0,900.0,400.0,100000000.0,100758800.0,11106.250000,318.750000,100.0,300.0,...,13137.500000,10468.750000,12218.750000,11025.000000,13337.500000,15175.000000,10018.750000,7556.250000,14387.500000,13943.750000


In [17]:
alameda_geoframe = pumas.loc[pumas.NAME10.str.contains('Alameda County')]
alameda_geoframe['internet'] = np.array(alameda_internet)

ValueError: Wrong number of items passed 228, placement implies 1

In [66]:
import mplleaflet
ax = alameda_geoframe.plot(edgecolor='black', alpha=0.75, figsize=(20, 10), column='internet', cmap='Blues', scheme='quantiles')
mplleaflet.display(fig=ax.figure)